In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode

/root/miniconda3/envs/internvl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)


def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose(
        [
            T.Lambda(lambda img: img.convert("RGB") if img.mode != "RGB" else img),
            T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
            T.ToTensor(),
            T.Normalize(mean=MEAN, std=STD),
        ]
    )
    return transform


def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float("inf")
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio


def dynamic_preprocess(
    image, min_num=1, max_num=12, image_size=448, use_thumbnail=False
):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j)
        for n in range(min_num, max_num + 1)
        for i in range(1, n + 1)
        for j in range(1, n + 1)
        if i * j <= max_num and i * j >= min_num
    )
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size
    )

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size,
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images


def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert("RGB")
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(
        image, image_size=input_size, use_thumbnail=True, max_num=max_num
    )
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

In [3]:
generation_config = dict(max_new_tokens=512, do_sample=False)

In [4]:
def chat_vlm(model,tokenizer,image_list,header):
    pixel_values_list = [load_image(image, max_num=12).to(torch.bfloat16).cuda() for image in image_list]
    pixel_values = torch.cat(pixel_values_list, dim=0)
    num_patches_list = [pv.size(0) for pv in pixel_values_list]
    
    image_placeholder = "<image>\n"*len(image_list)
    question = (
        f"以“{header}”为题，结合图片内容{image_placeholder}，写一篇小红书文案。"
    )
    response, history = model.chat(
        tokenizer,
        pixel_values,
        question,
        generation_config,
        num_patches_list=num_patches_list,
        history=None,
        return_history=True,
    )
    print(f"User: {question}\nAssistant: {response}")

## 微调前模型

In [5]:
path = "OpenGVLab/InternVL2_5-1B"
model = (
    AutoModel.from_pretrained(
        path,
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        use_flash_attn=True,
        trust_remote_code=True,
    )
    .eval()
    .cuda()
)
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

In [6]:
chat_vlm(
    model = model,
    tokenizer=tokenizer,
    image_list=[
        # "https://d.ifengimg.com/q100/img1.ugc.ifeng.com/newugc/20191218/12/wemedia/cdb8eda8463e7728ae2ccfbd0273292c947ec322_size885_w1620_h1080.jpg",
        # "https://img.pconline.com.cn/images/upload/upc/tx/itbbs/1404/29/c18/33701467_1398762909920_mthumb.jpg"
        "./assets/test/cdb8eda8463e7728ae2ccfbd0273292c947ec322_size885_w1620_h1080.jpg",
        "./assets/test/33701467_1398762909920_mthumb.jpg"
                ],
    header="今天来看大熊猫啦"
)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


User: 以“今天来看大熊猫啦”为题，结合图片内容<image>
<image>
，写一篇小红书文案。
Assistant: 标题：大熊猫的温馨时刻！

正文：
今天，我有幸和大家分享了我们可爱的熊猫宝宝们。你们是不是也喜欢这样的画面呢？你们知道吗？大熊猫是国家一级保护动物，它们的出现，让我们的世界更加美好。

🌿熊猫宝宝们在我们身边，它们的毛茸茸的身体，可爱的表情，让人心动。你们是不是也想和它们一起玩耍，一起探索这个世界呢？你们知道吗？大熊猫的毛皮柔软，它们的呼吸方式独特，让我们的生活更加美好。

#熊猫生活 #熊猫保护 #熊猫可爱 #熊猫探索

希望你们也能和我一样，喜欢这样的画面，喜欢这样的熊猫宝宝们。让我们一起保护这些可爱的小生命，让它们在我们的世界里，继续快乐地生活。

#熊猫 #熊猫保护 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫保护 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊猫可爱 #熊猫探索 #熊猫生活 #熊

## 微调后模型

In [7]:
import os
dir_path = os.getcwd()

import sys
sys.path.append(os.path.join(dir_path,"InternVL/internvl_chat"))

In [8]:
from internvl.model.internvl_chat import InternVLChatModel

################## 这里填写微调后的lora路径#####################
lora_path = os.path.join(dir_path,"InternVL/internvl_chat/work_dirs/internvl_chat_v2_5/internvl2_5_1b_dynamic_res_2nd_finetune_lora")
model = InternVLChatModel.from_pretrained(
    lora_path, low_cpu_mem_usage=True, torch_dtype=torch.bfloat16).eval()
tokenizer = AutoTokenizer.from_pretrained(lora_path, trust_remote_code=True)

if model.config.use_backbone_lora:
    model.vision_model.merge_and_unload()
    model.vision_model = model.vision_model.model
    model.config.use_backbone_lora = 0
if model.config.use_llm_lora:
    model.language_model.merge_and_unload()
    model.language_model = model.language_model.model
    model.config.use_llm_lora = 0
model.eval().cuda()

trainable params: 8,798,208 || all params: 638,496,128 || trainable%: 1.3779579255334184


InternVLChatModel(
  (vision_model): InternVisionModel(
    (embeddings): InternVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): InternVisionEncoder(
      (layers): ModuleList(
        (0-23): 24 x InternVisionEncoderLayer(
          (attn): InternAttention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (inner_attn): FlashAttention()
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (mlp): InternMLP(
            (act): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (norm2): LayerNorm((1024,), eps=1e-06, elementwi

In [9]:
chat_vlm(
    model = model,
    tokenizer=tokenizer,
    image_list=[
        # "https://d.ifengimg.com/q100/img1.ugc.ifeng.com/newugc/20191218/12/wemedia/cdb8eda8463e7728ae2ccfbd0273292c947ec322_size885_w1620_h1080.jpg",
        # "https://img.pconline.com.cn/images/upload/upc/tx/itbbs/1404/29/c18/33701467_1398762909920_mthumb.jpg"
        "./assets/test/cdb8eda8463e7728ae2ccfbd0273292c947ec322_size885_w1620_h1080.jpg",
        "./assets/test/33701467_1398762909920_mthumb.jpg"
        ],
    header="今天来看大熊猫啦"
)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


User: 以“今天来看大熊猫啦”为题，结合图片内容<image>
<image>
，写一篇小红书文案。
Assistant: 标题：今天来看大熊猫啦！🌿

文案：

大熊猫，憨态可掬的动物，是不是也让你心动了？今天带大家走进它们的自然栖息地，感受一下大熊猫的日常生活！

🌿 **自然栖息地**  
大熊猫喜欢在森林里活动，它们的家是郁郁葱葱的竹林和茂密的灌木丛。每天清晨，它们会在树下觅食，阳光透过树叶洒在它们身上，仿佛在为它们编织一幅美丽的画卷。午后，它们会在树下休息，享受着阳光和大自然的恩赐。

🐱 **可爱的小脸**  
大熊猫的可爱之处就在于它们那圆圆的大眼睛和柔软的毛发。它们的鼻子特别长，像一把大伞，可以伸出来呼吸新鲜空气。它们的耳朵竖起来，仿佛在倾听周围的动静，仿佛在和我们说悄悄话。

🌿 **食物的选择**  
大熊猫的食物非常丰富，包括竹子、山药、蘑菇等。它们喜欢在树上寻找食物，有时候还会在树洞里挖洞，寻找隐藏的美味。它们的饮食习惯非常独特，可以说是“杂食动物”！

大熊猫不仅可爱，还非常聪明，它们会用鼻子来嗅探食物，还会用嘴巴来咬食物。它们的智慧和耐心，让人惊叹不已。

大熊猫，你们是大自然的守护者，也是我们的朋友。下次如果有机会，一定要去它们的栖息地看看，感受一下它们的日常生活！🌿

#大熊猫 #熊猫 #自然栖息地 #熊猫生活 #自然观察 #动物摄影 #自然摄影 #大熊猫摄影
